# Meteorology



In [1]:
import os

HOME = os.environ['HOME']+"/Repositories/AirQualityCOVID/"

os.chdir(HOME+"src/Curation")

In [2]:
from datetime import date

import pandas as pd

from meteo.AEMET import DownloadAEMET
from meteo.apikey_file import apikey # import AEMET api key from file
from meteo import curation as curt

In [3]:
# Initialize AEMET API class with my api key
Aemet = DownloadAEMET(apikey=apikey)   

In [4]:
# ------------------------------
#      Air Quality sites
# ------------------------------
sites_AQ = pd.read_csv(HOME+"data/Curation/AirQuality/checked_sitesAQ.csv")
unique_sites_AQ =  sites_AQ.groupby(["site",
                                     "latitude",
                                     "longitude"]
                                   ).size().reset_index()

# Madrid, Santander, Barcelona
sites_lv = ["es0118a", "es1580a", "es1480a"]
unique_sites_AQ = unique_sites_AQ[unique_sites_AQ["site"].isin(sites_lv)]


In [5]:
# ------------------------------
#      Curation Variables
# ------------------------------

study_prd = [date(2013, 1, 1),    # start date
             date(2020, 12, 31)]  # end date
num_stations = 3
min_proportion = 0.8

folder = HOME+"data/Curation/AEMET/Values/"
removed_cl = ["sol", "horatmin", "horatmax", "horaracha",
              "horaPresMax", "horaPresMin"]

In [6]:
# ------------------------------
#      Curation Process
# ------------------------------
all_station_info = [curt.download_nearest_data(aemet=Aemet, siteAQ=row,
                                          stdy_prd=study_prd,
                                          remove_cl=removed_cl,
                                          folder=folder, n=num_stations,
                                          min_prop=min_proportion)
                    for i, row in unique_sites_AQ.iterrows()]


1/2: exito
2/2: exito
1/2: exito
2/2: exito
1/2: No hay datos que satisfagan esos criterios
1/2: exito
2/2: exito


In [9]:
# ------------------------------
#      Save Curated data
# ------------------------------
file_sv = HOME+"data/Curation/AEMET/sites_AEMET.csv"

chckd_st = pd.concat(all_station_info, 
                     axis=0, ignore_index=True)

chckd_st.head()

,latitud,provincia,altitud,indicativo,nombre,indsinop,longitud,dist,siteAQ
0,40.4119,MADRID,667,3195,"MADRID, RETIRO",08222,-3.67806,1.13715,es0118a
1,41.4183,BARCELONA,408,0200E,"BARCELONA, FABRA",,2.12417,3.26846,es1480a
2,43.4911,CANTABRIA,52,1111,SANTANDER,08023,-3.80056,3.45874,es1580a


### Guardar Datos en csv <a id="saveAQ"></a>

```python
chckd_st.to_csv(file_sv, index=False) 
```